In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import copy
import math
import pickle

sys.path.append("../../")

import numpy as np
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import meshio

import implicitmodules.torch as dm

#torch.set_num_threads(6)
torch.set_default_tensor_type(torch.FloatTensor)

In [ ]:
source = torch.zeros(8, 3)
source[0, 0] = -1.
source[0, 1] = -1.
source[0, 2] = -1.

source[1, 0] = -1.
source[1, 1] = 1.
source[1, 2] = -1.

source[2, 0] = -1
source[2, 1] = 1.
source[2, 2] = 1.

source[3, 0] = -1
source[3, 1] = -1.
source[3, 2] = 1.

source[4, 0] = 1.
source[4, 1] = -1.
source[4, 2] = -1.

source[5, 0] = 1.
source[5, 1] = 1.
source[5, 2] = -1.

source[6, 0] = 1
source[6, 1] = 1.
source[6, 2] = 1.

source[7, 0] = 1
source[7, 1] = -1.
source[7, 2] = 1.

mesh = meshio.read("../../data/sphere.stl")

source = torch.tensor(mesh.points).to(dtype=torch.get_default_dtype())
source = source[::5, :].contiguous()

target = source * torch.tensor([1., 1., 1.5])

%matplotlib qt5
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(source[:, 0].numpy(), source[:, 1].numpy(), source[:, 2].numpy(), marker='o')
plt.show()

In [ ]:
sigma0 = 0.2
nu0 = 0.01
implicit0 = dm.DeformationModules.ImplicitModule0.build_from_points(3, source.shape[0], sigma0, nu0, coeff=1., gd=source.view(-1).requires_grad_())

In [ ]:
model = dm.Models.ModelPointsRegistration([source], [implicit0], [dm.Attachment.EuclideanPointwiseDistanceAttachement()])
fitter = dm.Models.ModelFittingScipy(model, 1.)

costs = fitter.fit([target], 100, log_interval=10)